In [1]:
import pandas as pd
import numpy as np

from sklearn import linear_model
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_csv('./german_credit_data.csv', index_col=0)
df

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,67,male,2,own,NaN,little,1169,6,radio/TV,good
1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,49,male,1,own,little,NaN,2096,12,education,good
3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,53,male,2,free,little,little,4870,24,car,bad
...,...,...,...,...,...,...,...,...,...,...
995,31,female,1,own,little,NaN,1736,12,furniture/equipment,good
996,40,male,3,own,little,little,3857,30,car,good
997,38,male,2,own,little,NaN,804,12,radio/TV,good
998,23,male,2,free,little,little,1845,45,radio/TV,bad


In [3]:
df.dtypes

Age                  int64
Sex                 object
Job                  int64
Housing             object
Saving accounts     object
Checking account    object
Credit amount        int64
Duration             int64
Purpose             object
Risk                object
dtype: object

In [21]:
from sklearn.preprocessing import LabelEncoder

cat = ['Sex', 'Housing', 'Saving accounts', 'Checking account', 'Purpose', 'Risk']

le = LabelEncoder()

for c in cat:
    df[c] = le.fit_transform(df[c])
    

X, Y = df.iloc[:, :-1], df.iloc[:, -1]
train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size = 0.2, random_state = 100)

#train_X = scale(train_X)
#test_X = scale(test_X)

In [22]:
lasso = linear_model.Lasso()
lasso.fit(train_X, train_Y)

# Hyperparameter
alg = 'Lasso'

param_ls = {
    'alpha': np.logspace(0, 10, 100)
}

ls_cv = GridSearchCV(lasso, param_grid = param_ls, cv=10)
ls_cv.fit(train_X, train_Y)

lasso = ls_cv.best_estimator_
print(ls_cv.best_params_)

{'alpha': 1.0}


In [23]:
lasso = linear_model.Lasso(alpha=1)
lasso.fit(train_X, train_Y)

train_Y_hat = lasso.predict(train_X)
test_Y_hat = lasso.predict(test_X)
mse_train = mean_squared_error(train_Y, train_Y_hat)
mse_test = mean_squared_error(test_Y, test_Y_hat)

print(f'{alg} | Train MSE: {mse_train:.5f}, Test MSE: {mse_test:.5f}')
print(lasso.score(test_X, test_Y))

Lasso | Train MSE: 0.20658, Test MSE: 0.19874
0.014168540686615327


In [24]:
ridge = linear_model.Ridge()
ridge.fit(train_X, train_Y)

# Hyperparameter
alg = 'Ridge'

param_rg = {
    'alpha': np.logspace(0, 20, 100)
}

rg_cv = GridSearchCV(ridge, param_grid = param_rg, cv=10)
rg_cv.fit(train_X, train_Y)

ridge = rg_cv.best_estimator_
print(rg_cv.best_params_)

{'alpha': 104.76157527896652}


In [26]:
ridge = linear_model.Lasso(alpha=104.76157527896652)
ridge.fit(train_X, train_Y)

train_Y_hat = ridge.predict(train_X)
test_Y_hat = ridge.predict(test_X)
mse_train = mean_squared_error(train_Y, train_Y_hat)
mse_test = mean_squared_error(test_Y, test_Y_hat)

print(f'{alg} | Train MSE: {mse_train:.5f}, Test MSE: {mse_test:.5f}')
print(lasso.score(test_X, test_Y))

Ridge | Train MSE: 0.20796, Test MSE: 0.19914
0.014168540686615327
